In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, accuracy_score
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM
from sklearn.utils import class_weight

import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import pearsonr

from functions.nn import *

from fpdf import FPDF
import os

from collections import defaultdict

In [8]:
data = pd.read_csv('training_data/eurjpy_dataset_2025.csv')
data = data.drop(['Date_Time'], axis=1)

X = data.drop(['label'], axis=1)
y = data['label'].astype(int)

In [26]:
# Parameters
window_size = 5000
val_size = 1000
step = 1000

cost_per_trade = 1.5  # in pips

pip_value_per_standard_lot = 10 # Assuming EUR/USD and USD account (this is for a standard lot)
initial_account_balance = 10000.0
risk_per_trade_percentage = 0.01

close_prices = X['Close'].values
highs_full = X['High'].values
lows_full = X['Low'].values

# If any are 1-element tuples, extract the array
if isinstance(highs_full, tuple) and len(highs_full) == 1:
    highs_full = highs_full[0]
if isinstance(lows_full, tuple) and len(lows_full) == 1:
    lows_full = lows_full[0]
if isinstance(close_prices, tuple) and len(close_prices) == 1:
    close_prices = close_prices[0]

# Tracking
f1_per_window, acc_per_window, trade_per_window, profit_per_window = [], [], [], []
window_indices = []

current_account_balance_at_window_start = initial_account_balance

class_to_direction = {0: -1, 1: -1, 2: 0, 3: 1, 4: 1}

# Prediction horizon
steps = int(7)  # 7 candles
extra_steps = 0  # No extra steps for now

# window_length = 64

winning_trades = 0
losing_trades = 0

# Add tracking structures at the top
profit_per_class = defaultdict(float)
trades_per_class = defaultdict(int)

for i, start in enumerate(range(0, len(X) - window_size - val_size - steps, step)):

    # Standardize per window
    scaler = StandardScaler()
    scaler.fit(X[start : start + window_size])

    # Transform both the training data and the validation data
    X_scaled_train = scaler.transform(X[start : start + window_size])
    X_scaled_val = scaler.transform(X[start + window_size : start + window_size + val_size])
    train_X = X_scaled_train
    val_X = X_scaled_val

    val_y = y[start+window_size:start+window_size+val_size]
    val_y_cat = to_categorical(val_y, num_classes=5)

    train_y = y[start:start+window_size]
    train_y_cat = to_categorical(train_y, num_classes=5)

    input_features = train_X.shape[1]

    # train_X_seq, train_y_seq = create_lstm_sequences(train_X, train_y_cat, window_length)
    # val_X_seq, val_y_seq = create_lstm_sequences(val_X, val_y_cat, window_length)

    # 2. Optimize SL/TP on training window
    sl_tp_map = optimize_sl_tp_per_class(
        y=train_y,
        close_prices=close_prices,
        highs=highs_full,
        lows=lows_full,
        sl_values=[8, 10, 12, 15, 20],
        tp_values=[10, 12, 15, 20, 25],
        class_to_direction=class_to_direction,
        cost_per_trade=cost_per_trade
    )

    # 3. Estimate label horizon per class
    # Fui ver e isto varia bastante entre as windows por isso deixar
    avg_duration_by_class = estimate_avg_duration_per_class(
        y=train_y,
        close_prices=close_prices,
        highs=highs_full,
        lows=lows_full,
        sl_tp_map=sl_tp_map,
        class_to_direction=class_to_direction
    )

    '''# 4. Relabel training window using updated SL/TP and horizons
    train_y = relabel_data(
        X_window,
        sl_tp_map=sl_tp_map,
        avg_duration_by_class=avg_duration_by_class,
        class_to_direction=class_to_direction
    )
    train_y_cat = to_categorical(train_y, num_classes=5)'''

    cw = dict(enumerate(class_weight.compute_class_weight(
        class_weight='balanced', classes=np.unique(train_y), y=train_y)))

    # removed the validation set from the model training
    model = build_model_nn(input_features)
    model.fit(train_X, train_y_cat,
              epochs=30, batch_size=32,
              class_weight=cw,
              verbose=0)

    preds = np.argmax(model.predict(val_X, verbose=0), axis=1)
    # val_y_seq_labels = np.argmax(val_y, axis=1)

    f1 = f1_score(val_y, preds, average='weighted')
    acc = accuracy_score(val_y, preds)

    # Trade simulation
    val_start = start + window_size
    max_len = min(val_size, len(data) - val_start - steps)

    profit = 0.0
    trades = 0.0

    running_balance_in_window = current_account_balance_at_window_start

    # Fix tuple issue for close_prices and highs_full
    close_prices_arr = close_prices[0] if isinstance(close_prices, tuple) else close_prices
    highs_full_arr = highs_full[0] if isinstance(highs_full, tuple) else highs_full
    lows_full_arr = lows_full[0] if isinstance(lows_full, tuple) else lows_full

    # Use correct entry_prices and future_highs_seq
    entry_prices_arr = close_prices_arr[val_start:val_start + max_len]
    future_highs_seq_arr = [highs_full_arr[t:t+steps] for t in range(val_start, val_start + max_len)]
    future_lows_seq = [lows_full_arr[t:t+steps] for t in range(val_start, val_start + max_len)]

    for pred, entry, highs_seq, lows_seq in zip(preds[:max_len], entry_prices_arr, future_highs_seq_arr, future_lows_seq):
        direction = class_to_direction.get(pred, 0)
        if direction == 0:
            continue

        sltp = sl_tp_map.get(pred, {'sl': None, 'tp': None})
        if sltp['sl'] is None or sltp['tp'] is None:
            continue

        current_sl_pips = sltp['sl'] # ADDED: Get SL for lot size calculation
        current_tp_pips = sltp['tp'] # ADDED: Get TP for clarity

        # ADDED: Ensure SL is valid for lot size calculation
        if current_sl_pips is None or current_sl_pips <= 0:
            print(f"Warning: SL for pred {pred} is {current_sl_pips}. Skipping trade due to invalid SL for lot size calculation.")
            continue

        # Calculate monetary risk for this specific trade
        monetary_risk_for_this_trade = running_balance_in_window * risk_per_trade_percentage

        # Calculate the lot size multiplier required for this trade
        calculated_lot_size_multiplier = monetary_risk_for_this_trade / (current_sl_pips * pip_value_per_standard_lot)

        # Apply broker's minimum and maximum lot size constraints
        min_broker_lot_size = 0.01 # Example: Minimum micro lot
        max_broker_lot_size = 50.0 # Example: Maximum standard lots allowed
        

        calculated_lot_size_multiplier = min(max_broker_lot_size, calculated_lot_size_multiplier)
        calculated_lot_size_multiplier = round(calculated_lot_size_multiplier, 2)
        if calculated_lot_size_multiplier < min_broker_lot_size:
            continue
        
        '''# Use a fixed lot size per trade (e.g., 1 standard lot)
        fixed_lot_size = 1.0  # You can try other values like 0.1 or 0.5

        # Skip trades that fall outside broker limits
        if fixed_lot_size < min_broker_lot_size or fixed_lot_size > max_broker_lot_size:
            continue

        calculated_lot_size_multiplier = fixed_lot_size'''

        # Dynamic candle limit per class
        limit = avg_duration_by_class.get(pred) + extra_steps
        highs_limited = highs_seq[:limit]
        lows_limited = lows_seq[:limit]

        result, _ = simulate_trade(entry, highs_limited, lows_limited, direction, sltp['sl'], sltp['tp'])
        result -= cost_per_trade
        trades += 1

        # Count win/loss
        if result > 0:
            winning_trades += 1
        elif result < 0:
            losing_trades += 1

        trade_monetary_profit = result * pip_value_per_standard_lot * calculated_lot_size_multiplier
        profit += trade_monetary_profit
        running_balance_in_window += trade_monetary_profit

        # Track per class
        profit_per_class[pred] += trade_monetary_profit
        trades_per_class[pred] += 1

    # Log
    trade_per_window.append(trades)
    profit_per_window.append(profit)
    f1_per_window.append(f1)
    acc_per_window.append(acc)
    window_indices.append(i)

    current_account_balance_at_window_start = running_balance_in_window

    print(f"Window {i}: F1 = {f1:.3f}, Accuracy = {acc:.3f}, Profit = {profit:.2f}, Trades = {trades}, Current Balance: {current_account_balance_at_window_start:.2f}")

Window 0: F1 = 0.909, Accuracy = 0.939, Profit = 9665544.80, Trades = 1000.0, Current Balance: 9675544.80


KeyboardInterrupt: 

In [29]:
generate_model_report_pdf(
    steps,
    extra_steps,
    window_indices,
    f1_per_window,
    acc_per_window,
    profit_per_window, # Ensure this list contains monetary profits ($)
    trade_per_window,  # Ensure this list contains total trades for each window<
    initial_account_balance,
    # Parameters
    window_size,
    val_size,
    step,
    cost_per_trade,
    pip_value_per_standard_lot, # Corrected name for clarity
    risk_per_trade_percentage,  # Corrected name for clarity (e.g., 0.1 for mini lot)
    winning_trades,
    losing_trades,
    profit_per_class,
    trades_per_class,
    report_filename="model_timedata_risk_precentage_2025.pdf"
)

C:\Users\marti\AppData\Local\Temp\ipykernel_1180\3614619908.py:262: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 20)
C:\Users\marti\AppData\Local\Temp\ipykernel_1180\3614619908.py:263: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "Trading Model Performance Report", 0, 1, "C")
C:\Users\marti\AppData\Local\Temp\ipykernel_1180\3614619908.py:264: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "", 12)
C:\Users\marti\AppData\Local\Temp\ipykernel_1180\3614619908.py:265: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}", 0, 1, "C")
C:\Users\


Report generated successfully: model_timedata_risk_precentage_2025.pdf


In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM
from sklearn.utils import class_weight

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import os

def create_lstm_sequences(data, labels, window_length):
    X_seq, y_seq = [], []
    for i in range(len(data) - window_length):
        X_seq.append(data[i:i+window_length])
        y_seq.append(labels[i+window_length]) 
    return np.array(X_seq), np.array(y_seq)

def build_model_rnn(window_length, num_features, num_classes):
    model = Sequential([
        Input(shape=(window_length, num_features)),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_model_nn(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(5, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Has the limit in count
def simulate_trade(entry_price, highs, lows, direction, sl, tp):
    for i, (high, low) in enumerate(zip(highs, lows)):
        if direction == 1:
            if (high - entry_price) * 10_000 >= tp:
                return tp, i + 1
            elif (entry_price - low) * 10_000 >= sl:
                return -sl, i + 1
        elif direction == -1:
            if (entry_price - low) * 10_000 >= tp:
                return tp, i + 1
            elif (high - entry_price) * 10_000 >= sl:
                return -sl, i + 1
    final_price = highs[-1] if direction == 1 else lows[-1]
    result = (final_price - entry_price) * 10_000 * direction
    return result, len(highs)

def optimize_sl_tp_per_class(y, close_prices, highs, lows, sl_values, tp_values, class_to_direction, cost_per_trade):
    sl_tp_map = {}
    for cls in [0, 1, 3, 4]:
        best_profit = -np.inf
        best_pair = (12, 20)
        for sl in sl_values:
            for tp in tp_values:
                temp_profit = 0
                count = 0
                for idx, pred in enumerate(y):
                    if pred != cls:
                        continue
                    direction = class_to_direction.get(pred, 0)
                    if direction == 0:
                        continue
                    # Make sure we have enough future data
                    if idx + 7 > len(close_prices):
                        continue
                    entry = close_prices[idx]
                    highs_seq = highs[idx:idx+7]
                    lows_seq = lows[idx:idx+7]
                    outcome, _ = simulate_trade(entry, highs_seq, lows_seq, direction, sl, tp)
                    temp_profit += outcome
                    temp_profit -= cost_per_trade
                    count += 1
                if count > 0 and temp_profit > best_profit:
                    best_profit = temp_profit
                    best_pair = (sl, tp)
        sl_tp_map[cls] = {'sl': best_pair[0], 'tp': best_pair[1]}
    sl_tp_map[2] = {'sl': None, 'tp': None}  # no-trade
    return sl_tp_map

def estimate_avg_duration_per_class(y, close_prices, highs, lows, sl_tp_map, class_to_direction):
    duration_by_class = {0: [], 1: [], 3: [], 4: []}

    for idx, pred in enumerate(y):
        direction = class_to_direction.get(pred, 0)
        if direction == 0:
            continue
        sltp = sl_tp_map.get(pred, {'sl': None, 'tp': None})
        if sltp['sl'] is None or sltp['tp'] is None:
            continue
        # Make sure we have enough future data
        if idx + 12 > len(close_prices):
            continue
        entry = close_prices[idx]
        highs_seq = highs[idx:idx+12]
        lows_seq = lows[idx:idx+12]
        _, duration = simulate_trade(entry, highs_seq, lows_seq, direction, sltp['sl'], sltp['tp'])
        duration_by_class[pred].append(duration)

    avg_duration_by_class = {
        k: round(np.mean(v)) if v else 7 for k, v in duration_by_class.items()
    }
    return avg_duration_by_class


# not being used yet

def relabel_data(df, sl_tp_map, avg_duration_by_class, class_to_direction):
    relabeled = []
    for i in range(len(df)):
        row = df.iloc[i]
        label = row['label']
        direction = class_to_direction.get(label, 0)
        if direction == 0:
            relabeled.append(2)  # No-trade
            continue

        sltp = sl_tp_map.get(label, {'sl': None, 'tp': None})
        horizon = avg_duration_by_class.get(label)
        if sltp['sl'] is None or sltp['tp'] is None:
            relabeled.append(2)
            continue

        highs_seq = df['High'].iloc[i:i+horizon].values
        lows_seq = df['Low'].iloc[i:i+horizon].values
        if len(highs_seq) < horizon or len(lows_seq) < horizon:
            relabeled.append(2)
            continue

        result, _ = simulate_trade(row['Close'], highs_seq, lows_seq, direction, sltp['sl'], sltp['tp'])
        if result > 0:
            relabeled.append(label)
        else:
            relabeled.append(2)  # No-trade if neither hit
    return np.array(relabeled)


# see this last part of the result > 0 because then it's sus


def generate_model_report_pdf(
    steps,
    extra_steps,
    window_indices,
    f1_per_window,
    acc_per_window,
    profit_monetary_per_window, # Ensure this list contains monetary profits ($)
    trades_per_window,           # Ensure this list contains total trades for each window
    initial_account_balance,
    # Parameters
    window_size,
    val_size,
    step,
    cost_per_trade,
    pip_value_per_standard_lot, # Corrected name for clarity
    risk_per_trade_percentage,        # Corrected name for clarity (e.g., 0.1 for mini lot)
    winning_trades,
    losing_trades,
    profit_per_class,  # Dictionary with class labels as keys and total profit as values
    trades_per_class,  # Dictionary with class labels as keys and total trades as values
    report_filename
):
    # --- 1. Calculate Overall Statistics ---
    total_profit = np.sum(profit_monetary_per_window)
    total_trades = np.sum(trades_per_window)
    avg_f1 = np.mean(f1_per_window)
    avg_acc = np.mean(acc_per_window)

    # Maximum drawdown calculation
    cumulative_balance = [initial_account_balance]
    for p in profit_monetary_per_window:
        cumulative_balance.append(cumulative_balance[-1] + p)
    cumulative_balance_arr = np.array(cumulative_balance)
    peak = cumulative_balance_arr[0]
    max_drawdown_percentage = 0
    # Ensure cumulative_balance_arr has more than one element to avoid errors
    if len(cumulative_balance_arr) > 1:
        for balance in cumulative_balance_arr:
            if balance > peak:
                peak = balance # Update peak if new high
            # Drawdown for current balance relative to peak
            # Ensure peak is not zero to avoid division by zero
            drawdown = (peak - balance) / peak if peak != 0 else 0
            if drawdown > max_drawdown_percentage:
                max_drawdown_percentage = drawdown

    # --- 2. Generate Plots as PNG images ---
    # Set a style for better visualization
    sns.set_style("whitegrid")
    plt.rcParams.update({'font.size': 10})

    # Plot 1: Performance Metrics per Window (2x2 grid)
    fig1, axes1 = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))
    fig1.suptitle('Rolling Window Backtest Performance Metrics', fontsize=16)

    axes1[0, 0].plot(window_indices, profit_monetary_per_window, marker='o', linestyle='-', color='green', markersize=4)
    axes1[0, 0].set_title('Profit per Window (Dollars)')
    axes1[0, 0].set_xlabel('Window Index')
    axes1[0, 0].set_ylabel('Total Profit ($)') # Corrected label to Dollars
    axes1[0, 0].grid(True)
    axes1[0, 0].axhline(0, color='gray', linestyle='--', linewidth=0.8)

    axes1[0, 1].plot(window_indices, trades_per_window, marker='o', linestyle='-', color='blue', markersize=4) # Changed 'trade_per_window' to 'trades_per_window'
    axes1[0, 1].set_title('Number of Trades per Window')
    axes1[0, 1].set_xlabel('Window Index')
    axes1[0, 1].set_ylabel('Number of Trades')
    axes1[0, 1].grid(True)

    axes1[1, 0].plot(window_indices, f1_per_window, marker='o', linestyle='-', color='purple', markersize=4)
    axes1[1, 0].set_title('F1-score per Window (Weighted)')
    axes1[1, 0].set_xlabel('Window Index')
    axes1[1, 0].set_ylabel('F1-score')
    axes1[1, 0].grid(True)
    axes1[1, 0].set_ylim(0, 1)

    axes1[1, 1].plot(window_indices, acc_per_window, marker='o', linestyle='-', color='red', markersize=4)
    axes1[1, 1].set_title('Accuracy per Window')
    axes1[1, 1].set_xlabel('Window Index')
    axes1[1, 1].set_ylabel('Accuracy')
    axes1[1, 1].grid(True)
    axes1[1, 1].set_ylim(0, 1)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plot1_path = 'temp_metrics_plot.png'
    fig1.savefig(plot1_path)
    plt.close(fig1) # Close the figure to free up memory

    # Plot 2: Cumulative Profit
    fig2 = plt.figure(figsize=(12, 6))
    plt.plot(window_indices, cumulative_balance_arr[1:], color='darkgreen', linewidth=2) # [1:] because initial balance is at index 0
    plt.title('Cumulative Account Balance Over Windows')
    plt.xlabel('Window Index')
    plt.ylabel(f'Cumulative Balance ($)')
    plt.grid(True)
    plt.axhline(initial_account_balance, color='orange', linestyle='--', linewidth=0.8, label='Initial Balance')
    plt.axhline(np.max(cumulative_balance_arr), color='blue', linestyle='--', linewidth=0.8, label='All-time High')
    plt.axhline(np.min(cumulative_balance_arr), color='red', linestyle='--', linewidth=0.8, label='All-time Low')
    plt.legend()
    plot2_path = 'temp_cumulative_plot.png'
    fig2.savefig(plot2_path)
    plt.close(fig2)

    # --- 3. Create PDF Report ---
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    # Title Page
    pdf.set_font("Arial", "B", 20)
    pdf.cell(0, 10, "Trading Model Performance Report", 0, 1, "C")
    pdf.set_font("Arial", "", 12)
    pdf.cell(0, 10, f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}", 0, 1, "C")
    pdf.ln(5)

    # Parameters Section
    pdf.set_font("Arial", "B", 14)
    pdf.cell(0, 10, "1. Backtest Parameters", 0, 1, "L")
    pdf.set_font("Arial", "", 10)
    pdf.multi_cell(0, 7, f"""
    - Window Size: {window_size} candles
    - Validation Size: {val_size} candles
    - Step Size: {step} candles
    - Cost per Trade: {cost_per_trade} pips
    - Pip Value (Standard Lot): ${pip_value_per_standard_lot}
    - Risk per trade (e.g., 0.1 for Mini Lot): {risk_per_trade_percentage}
    - Initial Account Balance: ${initial_account_balance:,.2f}
    - Extra Steps in the simulate trade: {extra_steps}
    - Total Steps in the prediction: {steps}
    """)
    pdf.ln(5)

    # Overall Summary Statistics
    pdf.set_font("Arial", "B", 14)
    pdf.cell(0, 10, "2. Overall Performance Summary", 0, 1, "L")
    pdf.set_font("Arial", "", 10)
    pdf.multi_cell(0, 7, f"""
    - Total Profit: ${total_profit:,.2f}
    - Total Trades: {total_trades:,}
    - Average F1-score (Weighted): {avg_f1:.3f}
    - Average Accuracy: {avg_acc:.3f}
    - Maximum Drawdown: {max_drawdown_percentage * 100:.2f}%
    - Final Account Balance: ${cumulative_balance_arr[-1]:,.2f}
    - All-time High Balance: ${np.max(cumulative_balance_arr):,.2f}
    - All-time Low Balance: ${np.min(cumulative_balance_arr):,.2f}
    - Winning Trades: {winning_trades:,}
    - Losing Trades: {losing_trades:,}
    """)
    pdf.ln(5)

    # Profit per predicted class summary
    pdf.set_font("Arial", "B", 14)
    pdf.cell(0, 10, "3. Profit Per Predicted Class", 0, 1, "L")
    pdf.set_font("Arial", "", 10)

    for cls in sorted(profit_per_class.keys()):
        total_profit_cls = profit_per_class[cls]
        total_trades_cls = trades_per_class[cls]
        avg_profit_cls = total_profit_cls / total_trades_cls if total_trades_cls else 0
        pdf.cell(0, 7, f"Class {cls}: Trades = {total_trades_cls}, "
                       f"Total Profit = ${total_profit_cls:,.2f}, "
                       f"Avg Profit/Trade = ${avg_profit_cls:,.2f}", ln=1)

    pdf.ln(3)

    # Win rate
    total_classified_trades = winning_trades + losing_trades
    win_rate = (winning_trades / total_classified_trades) * 100 if total_classified_trades else 0
    pdf.set_font("Arial", "B", 14)
    pdf.cell(0, 10, "4. Risk Metrics", 0, 1, "L")
    pdf.set_font("Arial", "", 10)
    pdf.cell(0, 7, f"Win Rate: {win_rate:.2f}%", ln=1)

    # Sharpe ratio calculation
    returns = np.array(profit_monetary_per_window)
    avg_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = avg_return / std_return if std_return > 0 else 0
    pdf.cell(0, 7, f"Sharpe Ratio: {sharpe_ratio:.2f}", ln=1)

    pdf.ln(5)

    # Plots Section
    pdf.add_page() # Add a new page for plots
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, "3. Performance Visualizations", 0, 1, "L")
    pdf.ln(5)

    # Add Plot 1
    pdf.image(plot1_path, x=10, y=pdf.get_y(), w=180)
    pdf.ln(fig1.get_size_inches()[1] * 13) # Move cursor down after image

    # Add Plot 2
    pdf.image(plot2_path, x=10, y=pdf.get_y(), w=180)
    pdf.ln(fig2.get_size_inches()[1] * 10) # Move cursor down after image

    # Output PDF
    pdf.output(report_filename)

    # --- 4. Clean up temporary image files ---
    os.remove(plot1_path)
    os.remove(plot2_path)

    print(f"\nReport generated successfully: {report_filename}")